In [9]:
import os 
os.chdir('..')

import pandas as pd
import tellurium as te
from src.odbm.odbm_main import ModelBuilder


In [77]:
#Define DataFrames of species, reactions from input model
model_species = pd.read_csv('src/frenda_brenda/Files/SpeciesBaseMechanisms.csv')
model_rxns = pd.read_csv('src/frenda_brenda/Files/Reaction.csv')

model_rxns['Id'] = model_rxns['Enzyme']+'; ' +model_rxns['Substrates']+'; '+model_rxns['Products']
model_rxns.drop_duplicates(subset='Id', inplace=True)

In [78]:
#Generate and save model to txt file
modelfile = 'src/frenda_brenda/Files/240226MC_FULL.txt'
myModel = ModelBuilder(model_species, model_rxns)
myModel.saveModel(modelfile)

r = []
while not r:
    try:
        r  = te.loada(myModel.compile())
    except Exception as e:
        myModel.addSpecies(str(e).split("'")[1], 0.001)
        print(str(e).split("'")[1])

s = r.simulate(0,10)

Error: CVODE Error: CV_CONV_FAILURE, Module: CVODES, Function: CVode, Message: At t = 0 and h = 1.76967e-19, the corrector convergence test failed repeatedly or with |h| = hmin.


RuntimeError: CVODE Error: CV_CONV_FAILURE: Convergence test failures occurred too many times (= MXNCF = 10) during one internal timestep or occurred with |h| = hmin.; In virtual double rr::CVODEIntegrator::integrate(double, double)

In [ ]:
import os
os.chdir('/mmfs1/gscratch/cheme/dalba/repos/ECFERS')

: 

In [84]:
r.exportToSBML('models/240226MC_FULL.sbml')

In [79]:
import numpy as np
for k in ['Km','Kcat_F','Kcat_R','Ki']:
    labels = [K for K in r.getGlobalParameterIds() if k in K]
    mean = np.nanmean([r[k] for k in labels])
    for k in labels:
        r[k] = mean